In [3]:
import pandas as pd

df= pd.read_csv("/content/MachineLearningRating_v4.csv")
df.head()

<ipython-input-3-1301563624>:3: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df= pd.read_csv("/content/MachineLearningRating_v4.csv")


,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims,LossRatio,HadClaim
0,145249,12827,2015-03-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0,0.0,0
1,145249,12827,2015-05-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0,0.0,0
2,145255,12827,2015-05-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0,0.0,0
3,145247,12827,2015-01-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Third Party,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,3.256435,0.0,0.0,0
4,145247,12827,2015-04-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Third Party,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,50.474737,0.0,0.0,0


In [4]:
# Clean up any edge cases
df = df[(df['TotalPremium'] > 0) & (df['TotalClaims'] >= 0)]

# Feature Engineering
df['HadClaim'] = (df['TotalClaims'] > 0).astype(int)
df['Margin'] = df['TotalPremium'] - df['TotalClaims']
df['LossRatio'] = df['TotalClaims'] / df['TotalPremium']


**Hypothesis 1 Risk Differences Across Provinces**

In [6]:
# Clean up any edge cases
df = df[(df['TotalPremium'] > 0) & (df['TotalClaims'] >= 0)]

# Feature Engineering
df['HadClaim'] = (df['TotalClaims'] > 0).astype(int)
df['Margin'] = df['TotalPremium'] - df['TotalClaims']
df['LossRatio'] = df['TotalClaims'] / df['TotalPremium']

province_claims = df.groupby('Province').agg(
    n=('HadClaim', 'count'),
    claims=('HadClaim', 'sum')
)
province_claims['ClaimFrequency'] = province_claims['claims'] / province_claims['n']
province_claims.sort_values(by='ClaimFrequency', ascending=False)


,n,claims,ClaimFrequency
Province,,,
Gauteng,240781,1243,0.005162
KwaZulu-Natal,111896,453,0.004048
Mpumalanga,31663,125,0.003948
North West,89799,334,0.003719
Western Cape,96757,356,0.003679
Limpopo,18009,66,0.003665
Eastern Cape,19694,47,0.002387
Northern Cape,3643,8,0.002196
Free State,5932,9,0.001517


In [7]:
import scipy.stats as stats
from scipy.stats import chi2_contingency

# Create contingency table
contingency = pd.crosstab(df['Province'], df['HadClaim'])

# Perform Chi-Squared test
chi2, p, dof, expected = chi2_contingency(contingency)
print(f"Chi-squared Test Statistic: {chi2}")
print(f"P-value: {p}")

if p < 0.05:
    print("Reject the null hypothesis — there are significant risk differences across provinces.")
else:
    print("Fail to reject the null hypothesis — no significant risk differences across provinces.")


Chi-squared Test Statistic: 93.69521170128786
P-value: 8.248012599642686e-17
Reject the null hypothesis — there are significant risk differences across provinces.


**Hypothesis 2: Risk Differences Across Zip Codes**

In [8]:
# Filter zip codes with enough samples to avoid noise
zip_counts = df['PostalCode'].value_counts()
valid_zips = zip_counts[zip_counts >= 30].index  # adjust threshold if needed
df_zip = df[df['PostalCode'].isin(valid_zips)]

# Chi-squared test
contingency_zip = pd.crosstab(df_zip['PostalCode'], df_zip['HadClaim'])
chi2_zip, p_zip, dof_zip, expected_zip = chi2_contingency(contingency_zip)

print(f"Chi-squared Test Statistic (Zip): {chi2_zip}")
print(f"P-value (Zip): {p_zip}")

if p_zip < 0.05:
    print("Reject the null hypothesis — significant risk differences between zip codes.")
else:
    print("Fail to reject the null hypothesis — no significant risk differences between zip codes.")


Chi-squared Test Statistic (Zip): 1133.666088329455
P-value (Zip): 1.2390975132821503e-15
Reject the null hypothesis — significant risk differences between zip codes.


**Hypothesis 3: Margin Differences Between Zip Codes**

In [9]:
from scipy.stats import f_oneway

# Group by PostalCode
zip_groups = [group['Margin'] for name, group in df_zip.groupby('PostalCode')]

# One-Way ANOVA
f_stat, p_anova = f_oneway(*zip_groups)

print(f"ANOVA F-statistic: {f_stat}")
print(f"P-value: {p_anova}")

if p_anova < 0.05:
    print("Reject the null hypothesis — significant margin differences between zip codes.")
else:
    print("Fail to reject the null hypothesis — no significant margin differences between zip codes.")


ANOVA F-statistic: 0.9093487209394084
P-value: 0.9662326410809475
Fail to reject the null hypothesis — no significant margin differences between zip codes.


**Hypothesis 4: Risk Differences Between Women and Men**

In [10]:
# Gender counts
gender_counts = df['Gender'].value_counts()
valid_genders = gender_counts[gender_counts >= 30].index
df_gender = df[df['Gender'].isin(valid_genders)]

# Claims by gender
claims_by_gender = df_gender.groupby('Gender')['HadClaim'].agg(['sum', 'count'])
group1, group2 = claims_by_gender.iloc[0], claims_by_gender.iloc[1]

# Proportions
import statsmodels.api as sm

z_test = sm.stats.proportions_ztest([group1['sum'], group2['sum']], [group1['count'], group2['count']])
print(f"Z-statistic: {z_test[0]}")
print(f"P-value: {z_test[1]}")

if z_test[1] < 0.05:
    print("Reject the null hypothesis — significant risk difference between genders.")
else:
    print("Fail to reject the null hypothesis — no significant risk difference between genders.")


Z-statistic: -3.3837923084775112
P-value: 0.000714920610918459
Reject the null hypothesis — significant risk difference between genders.


**Summary**

In [11]:
summary = pd.DataFrame({
    'Hypothesis': [
        'Risk across Provinces',
        'Risk across Zip Codes',
        'Margin across Zip Codes',
        'Risk between Genders'
    ],
    'P-Value': [p, p_zip, p_anova, z_test[1]],
    'Rejected H₀?': [
        p < 0.05,
        p_zip < 0.05,
        p_anova < 0.05,
        z_test[1] < 0.05
    ]
})

summary


,Hypothesis,P-Value,Rejected H₀?
0,Risk across Provinces,8.248013e-17,True
1,Risk across Zip Codes,1.239098e-15,True
2,Margin across Zip Codes,9.662326e-01,False
3,Risk between Genders,7.149206e-04,True


## Hypothesis Testing Summary

This section presents the results of statistical hypothesis tests conducted to validate whether certain features influence risk or margin in the insurance portfolio.

---

### Summary Table

| Hypothesis                          | P-Value        | Reject H₀? | Interpretation                                                                 |
|-------------------------------------|----------------|------------|---------------------------------------------------------------------------------|
| Risk across Provinces               | 8.25e-17       |  Yes     | Risk significantly varies by province. Supports region-specific premium pricing. |
| Risk across Zip Codes               | 1.24e-15       |  Yes     | Zip code segmentation is statistically valid. Enables hyper-local pricing.      |
| Margin across Zip Codes             | 0.966          |  No      | No significant margin difference between zip codes. Profitability is uniform.   |
| Risk between Genders                | 7.15e-04       |  Yes     | Statistically significant risk difference between genders. Requires ethical consideration. |

---

###  Interpretation & Recommendations

1. **Province-Based Risk Adjustment**  
   There is strong evidence of risk differences across provinces. Provinces with higher loss ratios (e.g., Gauteng) may warrant increased premiums or risk-adjusted underwriting.

2. **Zip Code Segmentation**  
   Risk varies meaningfully at the zip code level, suggesting opportunities for granular, localized pricing strategies. However, margins are not significantly different across zip codes, indicating uniform profitability.

3. **Gender-Based Risk**  
   A significant difference in claim frequency exists between genders. While statistically valid, incorporating gender into pricing decisions should be carefully evaluated due to regulatory and ethical considerations.

---

These insights will guide the design of segmentation strategies and support the development of targeted pricing models in Task 4.
